In [ ]:
import logging
import sys

from neo4j import GraphDatabase

class App:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        # Don't forget to close the driver connection when you are finished with it
        self.driver.close()

    @staticmethod
    def enable_log(level, output_stream):
        handler = logging.StreamHandler(output_stream)
        handler.setLevel(level)
        logging.getLogger("neo4j").addHandler(handler)
        logging.getLogger("neo4j").setLevel(level)

    #Function to query recommendations based on content filtering
    @staticmethod
    def _collabFiltering(tx):
        query = (
            "MATCH(u:User{id: "116878172175502198628"})-[:likes]->(s:Establishment)-[:has]->(c:Cuisine)<-[:has]-(z:Establishment) "
            "WHERE NOT EXISTS ((u)-[:likes]->(z)) "
            "WITH s, z "
            "MATCH (s)-[:has]->(sc:Cuisine) "
            "WITH s, z, COLLECT(DISTINCT id(sc)) AS s1, COLLECT(DISTINCT sc.cuisine) AS c1 "
            "MATCH (z)-[:has]->(zc:Cuisine) "
            "WITH s, z, s1, c1, COLLECT(DISTINCT id(zc)) AS s2, COLLECT(DISTINCT zc.cuisine) AS c2 "
            "WITH s, z, s1, s2, c1, c2 "
            "RETURN s.name as UserPreference, z.name as Recommended, c1 as UserPreferenceCuisine, c2 as RecommendedCuisine, gds.alpha.similarity.jaccard(s1,s2) AS similarity ORDER BY similarity DESC "
        )
        result = tx.run(query)
        return [{'UserPreference': row['UserPreference'],  'Recommended': row['Recommended'],  'UserPreferenceCuisine': row['UserPreferenceCuisine'], 'RecommendedCuisine': row['RecommendedCuisine'],
                                            'similarity': row['similarity']} for row in result]

    
    def collabFiltering(self):
        with self.driver.session() as session:
            results = session.write_transaction(self._find_conference_communities)
            print("Displaying content-based recommendations: \n")
            #fix to display less results
            for row in results[:5]:
                print(row)

    
    
if __name__ == "__main__":
    bolt_url = "neo4j://localhost:7687"
    user = "neo4j"
    password = "password123"
    App.enable_log(logging.INFO, sys.stdout)
    app = App(bolt_url, user, password)
    #app.collabFiltering()
    app.close()